# Introduction

*Saccharomyces cerevisiae* est un blablabla et on va étudier son réseau de régulation génétique en utilisant des outils de la théorie de graphe et de python library **networkx**. 

# Exploration and characterization of the gene regulatory network

In [1]:
import networkx as ntx
import matplotlib as mp
import numpy as np
import pandas as pd
ntx.__version__

'2.1'

Le fichier *GRN_edges_S_cerevisiae.txt* contient les liens entre les facteurs de transcription et les gènes. Il va nous servir pour créer notre graphe. On enlève la première colonne qui ne nous rajoute pas d'information supplémentaire.

In [2]:
netw = pd.read_csv("GRN_edges_S_cerevisiae.txt", sep = ',', header=0)
netw = netw.drop('Unnamed: 0', axis=1)
netw.head()

,transcription_factor,target_gene
0,G15,G1
1,G98,G1
2,G109,G1
3,G22,G9
4,G211,G11


In [3]:
G = ntx.Graph()
G = ntx.from_pandas_edgelist(netw, 'transcription_factor', 'target_gene')

On affiche le network en utilisant la commande *draw()*. Ce graphique ne nous apporte guère d'information et on doit alors trouver un autre moyen pour afficher le graphe.

In [4]:
ntx.draw(G)

On étudie le réseeau en calculant plusieurs paramètres.

In [5]:
ntx.average_clustering(G)
ntx.betweenness_centrality(G)
G.degree()

DegreeView({'G15': 84, 'G1': 122, 'G98': 82, 'G109': 68, 'G22': 76, 'G9': 1, 'G211': 62, 'G11': 42, 'G41': 219, 'G12': 1, 'G13': 1, 'G68': 54, 'G14': 35, 'G87': 114, 'G111': 78, 'G273': 48, 'G45': 56, 'G112': 102, 'G168': 19, 'G254': 6, 'G76': 164, 'G19': 15, 'G61': 44, 'G20': 6, 'G224': 120, 'G319': 95, 'G21': 7, 'G95': 43, 'G101': 55, 'G212': 54, 'G138': 199, 'G23': 1, 'G25': 9, 'G27': 1, 'G29': 2, 'G121': 56, 'G30': 4, 'G139': 27, 'G227': 27, 'G37': 9, 'G47': 20, 'G274': 26, 'G298': 70, 'G43': 3, 'G160': 97, 'G223': 78, 'G44': 1, 'G152': 109, 'G48': 3, 'G52': 11, 'G172': 38, 'G53': 2, 'G55': 2, 'G58': 3, 'G62': 29, 'G63': 1, 'G65': 13, 'G66': 11, 'G71': 78, 'G104': 108, 'G143': 64, 'G205': 13, 'G171': 64, 'G70': 3, 'G72': 1, 'G74': 1, 'G78': 2, 'G94': 32, 'G79': 1, 'G80': 1, 'G81': 1, 'G82': 3, 'G85': 6, 'G88': 1, 'G90': 1, 'G97': 48, 'G86': 13, 'G117': 66, 'G100': 1, 'G219': 72, 'G102': 17, 'G105': 1, 'G107': 9, 'G108': 1, 'G110': 32, 'G113': 10, 'G136': 46, 'G233': 11, 'G247': 16,

# Community detection

### Girvan–Newman algorithm

L'algorithme de Girvan–Newman est une méthode qui sert à trouver des communautés dans des graphes complexes.
Le but de cette méthode est d'identifier les edges qui se trouvent probablement entre les communauté, de les enlever et donc de récuperer les communautés individuelles.

L'algorhitme procéde de cette manière :
- calculer la *betweennes* de tous les edges du graphe
- enlever les edges ayant la *betweennes* plus élevée
- recalculer la *betweennes* des edges concernés
- refaire les étapes 2 et 3

En effet, les premiers edges à enlever auront la plus grande *betweennes* et donc seront beaucoup utilisés pour effectuer le passage le plus court entre 2 noeuds du graphe. Ce sont alors bien les edges qui servent comme des "ponts" entre les différentes communautés et sont beaucoup parcourus pour passer d'un côté du graphe à l'autre.

In [6]:
import itertools

k = 2
comp = ntx.algorithms.community.centrality.girvan_newman(G)

In [9]:
limited = itertools.takewhile(lambda c: len(c) <= k, comp)
for communities in limited:
    print(tuple(comm for comm in communities)) 

###  Louvain algorithm 

In [8]:
ntx.community.best_partition(G)

AttributeError: module 'networkx.algorithms.community' has no attribute 'best_partition'

# Conclusion